# Tugas 1 : Crawling Berita

**Pengertian Crawling**


Crawling adalah proses otomatis yang dilakukan oleh program komputer, yang disebut "crawler" atau "spider," untuk menelusuri dan mengumpulkan data dari situs web secara sistematis. Proses ini biasanya dimulai dengan mengakses halaman web tertentu dan kemudian mengikuti tautan (link) yang ada di dalamnya untuk mengakses halaman-halaman lain. Crawler mengunjungi halaman-halaman tersebut, mengumpulkan informasi seperti teks, gambar, dan metadata, dan menyimpan data tersebut untuk keperluan lebih lanjut, seperti pengindeksan di mesin pencari atau analisis data.

Crawling merupakan bagian penting dari proses web scraping dan pengindeksan mesin pencari, di mana mesin pencari seperti Google menggunakan crawler untuk mengumpulkan informasi dari seluruh web agar dapat menyediakan hasil pencarian yang relevan kepada pengguna.

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, min_articles_per_category):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len([k for k in kategori_list if k == kategori]) >= min_articles_per_category:
            return  # Menghentikan proses jika jumlah artikel sudah mencapai minimum yang diinginkan

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        judul.append(title)
        tanggal.append(date)
        isi.append(content)
        url_list.append(link)
        kategori_list.append(kategori)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = [
    "https://health.detik.com/berita-detikhealth/indeks",
    "https://travel.detik.com/travel-news/indeks"
]
categories = [
    "Kesehatan",
    "Pariwisata"
]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []
kategori_list = []

# Batas minimal artikel per kategori
min_articles_per_category = 50

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < min_articles_per_category:
        url = f"{base_url}/{page}"
        get_data(url, category, min_articles_per_category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman
        page += 1

# Membuat dataframe dari list data
df = pd.DataFrame({"judul": judul, "tanggal": tanggal, "isi": isi, "kategori": kategori_list})

# Menyimpan dataframe ke file csv
df.to_csv("data_berita_detik.csv", index=False)

# Menampilkan dataframe
print(df)

Badan Kurus Tak Berarti Bebas dari Kolesterol Tinggi, Dokter Ungkap Faktanya
Budaya Kerja Toksik di Jepang Picu 54 Karyawan Tewas Kelelahan
Menyoal Serangan Jantung yang Dialami Faisal Basri Sebelum Meninggal Dunia
Nyeri Kronis yang Bikin Paus Fransiskus Tak Bisa Lama-lama Tanpa Kursi Roda
Perundungan PPDS Tak Cuma Terjadi di Undip, Begini Kata Kemenkes
Dokter Ungkap 'Biang Kerok' Kolesterol Tinggi di Usia Muda, Ternyata gegara Ini
Update Kasus Mpox di RI, Kemenkes Temukan Suspek Baru di Jakarta dan Bali
5 Olahraga yang Efektif Bakar Kalori, Bantu Enyahkan Perut Buncit Lebih Cepat
RS Medistra Bantah Larangan Hijab, Beberkan Bukti Tak Ada Diskriminasi
Kata WHO soal RI Bisa Jadi Sudah Kemasukan Varian 'Ganas' Mpox
Babak Baru Bullying Dokter, Keluarga dr ARL Polisikan Senior PPDS Undip!
Sederet Gejala Kolesterol Tinggi yang Bisa Muncul pada Pria
Pengakuan Dokter Pasok Ketamine buat Matthew Perry yang Sebabkan Kematian
RS Medistra Bantah Larang Nakes Pakai Hijab: 30 Persen Pegawai Kami Ber

In [7]:
df=pd.read_csv("data_berita_detik.csv")
df

,judul,tanggal,isi,kategori
0,Badan Kurus Tak Berarti Bebas dari Kolesterol ...,"Kamis, 05 Sep 2024 11:03 WIB",Jakarta - Tak sedikit yang beranggapan bahwa o...,Kesehatan
1,Budaya Kerja Toksik di Jepang Picu 54 Karyawan...,"Kamis, 05 Sep 2024 10:38 WIB",Jakarta -\n \n Istilah 'bekerja ...,Kesehatan
2,Menyoal Serangan Jantung yang Dialami Faisal B...,"Kamis, 05 Sep 2024 10:03 WIB",Jakarta - Faisal Basri meninggal dunia. Ekonom...,Kesehatan
3,Nyeri Kronis yang Bikin Paus Fransiskus Tak Bi...,"Kamis, 05 Sep 2024 09:37 WIB",Jakarta - Pemimpin tertinggi Gereja Katolik se...,Kesehatan
4,"Perundungan PPDS Tak Cuma Terjadi di Undip, Be...","Kamis, 05 Sep 2024 08:35 WIB",Jakarta - Aksi bullying atau perundungan di ma...,Kesehatan
...,...,...,...,...
95,Luhut soal Penginapan dan Mobil untuk Paus yan...,"Rabu, 04 Sep 2024 05:49 WIB",Jakarta - Pemimpin Tertinggi Gereja Katolik Se...,Pariwisata
96,"Iwh, Pekerja Speedboat Thailand Pamer BAB ke L...","Rabu, 04 Sep 2024 05:01 WIB",Jakarta - Warga Thailand dibuat jijik dengan k...,Pariwisata
97,"Ogah Bayar Layanan, 3 Turis Palestina Keroyok ...","Selasa, 03 Sep 2024 21:05 WIB",Taba - Seorang staf hotel di Mesir tewas usai ...,Pariwisata
98,"Licik, Keluarga Taruh Kecoa di Piring Agar Mak...","Selasa, 03 Sep 2024 20:37 WIB",Jakarta - Keluarga asal Meksiko dituduh melaku...,Pariwisata
